In [ ]:
print('hiii')

In [3]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle 

In [4]:
#load datset
data = pd.read_csv("IMDB Dataset.csv")

In [5]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
data.replace({"sentiment": {"positive":1,"negative":0}},inplace=True)

C:\Users\PERIYASAMY.M\AppData\Local\Temp\ipykernel_16908\3714596480.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment": {"positive":1,"negative":0}},inplace=True)


In [7]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [8]:
train_data,test_data=train_test_split(data,test_size=0.2,random_state=0)

In [11]:
#tokenize text data 
tokenizer=Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])


In [12]:
#save the tokonzer
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer,f)

In [14]:
#prepare training and testing data 
x_train= pad_sequences(tokenizer.texts_to_sequences(train_data["review"]),maxlen=200)
x_test= pad_sequences(tokenizer.texts_to_sequences(test_data["review"]),maxlen=200)
y_train= train_data["sentiment"]
y_test= test_data["sentiment"]


In [15]:
#build the model 
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

c:\Users\PERIYASAMY.M\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [16]:
#compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


In [17]:
#train the model 
model.fit(x_train,y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 94s 179ms/step - accuracy: 0.7141 - loss: 0.5403 - val_accuracy: 0.8589 - val_loss: 0.3372
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 84s 169ms/step - accuracy: 0.8601 - loss: 0.3350 - val_accuracy: 0.8282 - val_loss: 0.3819
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 84s 168ms/step - accuracy: 0.8597 - loss: 0.3337 - val_accuracy: 0.8627 - val_loss: 0.3240
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 86s 172ms/step - accuracy: 0.8909 - loss: 0.2662 - val_accuracy: 0.8692 - val_loss: 0.3204
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 221s 442ms/step - accuracy: 0.9085 - loss: 0.2338 - val_accuracy: 0.8572 - val_loss: 0.3375


In [18]:
#Evualate the model
loss, accuracy = model.evaluate(x_test,y_test)
print(f"Test Loss:{loss}")
print(f"Test_accuaracy:{accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 31ms/step - accuracy: 0.8511 - loss: 0.3539
Test Loss:0.3557496964931488
Test_accuaracy:0.849399983882904


In [19]:
#save the model 
model.save("model.h5")